# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Aluno**: Vitor Silveira

**RA**: 802318


---
### Análise exploratória

Nesta seção, deve ser feita a leitura da base de dados e todas as análises necessárias para interpretar e analisar os dados, tais como:
* Significado de cada atributo
* Medidas descritivas
* Gráficos

In [ ]:
import numpy as np 
import pandas as pd 
import os

diretorio = 'data/raw_data'

arquivos_csv = [arquivo for arquivo in os.listdir(diretorio) if arquivo.startswith('new') and arquivo.endswith('.csv')]

dados_concatenados = pd.DataFrame()

for arquivo in arquivos_csv:
   print(arquivo)
   caminho_arquivo = os.path.join(diretorio, arquivo)
   dados = pd.read_csv(caminho_arquivo)
   dados_concatenados = pd.concat([dados_concatenados, dados], ignore_index=True)

alldata = dados_concatenados
dados_concatenados.to_csv(f'data/pre_processed_data/all_news.csv', index=False)

In [ ]:
train_csv = pd.read_csv("data/raw_data/train.csv")                    # É o Y (com os labels)

test_csv = pd.read_csv("data/raw_data/test.csv")                      # Teste

sample_sub = pd.read_csv("data/raw_data/sample_submission.csv")       # Submissão

alldata = pd.read_csv("data/pre_processed_data/all_news.csv")    

In [ ]:
test = test_csv.merge(alldata, on = "id")
test

In [ ]:
joinned_train = train_csv.merge(alldata, on = "id")
joinned_train

---
### Pré-processamento

Nesta seção, as funções da etapa de pré-processamento dos dados devem ser implementadas e aplicadas (se necessário).

In [ ]:
joinned_train.dropna(inplace = True)
joinned_train.drop(columns = [ "id", "date" ], inplace = True)

In [ ]:
test.drop(columns = [ "id", "date" ], inplace = True)

In [ ]:
joinned_train["title_content"] = joinned_train["title"] + " " + joinned_train["content"]
test["title_content"] = test["title"] + " " + test["content"]

joinned_train.drop(columns = [ "title", "content" ], inplace = True)
test.drop(columns = [ "title", "content" ], inplace = True)

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

rm_punctuation = string.punctuation

stopw = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopw])

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', rm_punctuation))

def to_lowercase(text):
    return text.lower()

def preprocess_text(text):
    if pd.isnull(text):
        return ""

    text = remove_punctuation(text)

    text = remove_stopwords(text)

    text = to_lowercase(text)

    return text

In [ ]:
joinned_train["title_content"] = joinned_train["title_content"].apply(lambda text: preprocess(text))
test["title_content"] = test["title_content"].apply(lambda text: preprocess(text))

---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais padrões e testando diferentes modelos.

In [ ]:
X = joinned_train["title_content"].astype(str)
y_train = joinned_train["label"]
X_test = test["title_content"].apply(lambda x: np.str_(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer(max_features = 8000)
x_train = vectorization.fit_transform(X)
X_test_vec = vectorization.transform(X_test)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(objective='binary:logistic')
xgb.fit(x_train, y_train)
predXGB = xgb.predict_proba(X_test_vec)
XGBscore = xgb.score(x_train, y_train)
print(XGBscore)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs', max_iter=1000).fit(x_train, y_train)
print("passo 1")
predRL = clf.predict_proba(X_test_vec)
print("passo 2")
LRscore = clf.score(x_train, y_train)
print(LRscore)

In [ ]:
sample_sub = pd.read_csv("data/raw_data/sample_submission.csv")   
sample_sub["label"] = predXGB[:, 1] 
sample_sub.set_index("id").to_csv("default-xgb-alldata.csv")

---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos através de tabelas e gráficos, comparados e profundamente analisados.